In [36]:
# - Decide if plotting totals or anomalies
# in main notebook code
#totanomnow = 'tot' # choose: tot, anom

In [ ]:
# - Define constant plot params
stipsizenow = 10
stipmarknow = 'o'
stipfacecolnow = 'none'
stipedgeltcolnow = 'lightgray'
stipewnow = 0.8 # marker edge width
signifstipnow = 1
eezfcnow = 'none'
eezlcnow = 'silver'
eezlsnow = '-'
eezlwnow = 1.5

In [ ]:
# - Define subplot-variable plot params
seasonnamesnow = ['Winter', 'Spring', 'Summer', 'Fall']
seasonsnow = ['DJF', 'MAM', 'JJA', 'SON']
nseas = 4; nvars = 3
mapsnow = nseas*nvars*[None]
pvsnow = nseas*nvars*[None] 
ptfsnow = nseas*nvars*[None] 
if totanomnow=='anom':
    cmnow = cm.PuOr
    stipedgedkcolnow = 'darkslategray'
    vmaxsnow = 4*[12, 4, 0.12]
    vminsnow = -1*np.asarray(vmaxsnow)
    ploteezsnow = 12*[0]
    signifstipsnow = 12*[1]
    for iseas in range(nseas):
        mapsnow[3*iseas] = skj_cp_tot_seas.sel(season=seasonsnow[iseas])-skj_cp_tot_mean 
        pvsnow[3*iseas] = skj_cp_tot_seas_kw_pval
        ptfsnow[3*iseas] = skj_cp_tot_seas_kw_ptf
        mapsnow[3*iseas+1] = bet_cp_tot_seas.sel(season=seasonsnow[iseas])-bet_cp_tot_mean 
        pvsnow[3*iseas+1] = bet_cp_tot_seas_kw_pval
        ptfsnow[3*iseas+1] = bet_cp_tot_seas_kw_ptf
        mapsnow[3*iseas+2] = bettoskj_cp_tot_seas.sel(season=seasonsnow[iseas])-bettoskj_cp_tot_mean 
        pvsnow[3*iseas+2] = bettoskj_cp_tot_seas_kw_pval
        ptfsnow[3*iseas+2] = bettoskj_cp_tot_seas_kw_ptf
elif totanomnow=='tot':
    cmnow = cm.viridis
    stipedgedkcolnow = 'lightgray'
    vmaxsnow = 4*[60, 15, 0.32]
    vminsnow = 0*np.asarray(vmaxsnow)
    ploteezsnow = 12*[1]
    signifstipsnow = 12*[0]
    for iseas in range(nseas):
        mapsnow[3*iseas] = skj_cp_tot_seas.sel(season=seasonsnow[iseas])
        pvsnow[3*iseas] = skj_cp_tot_seas_kw_pval
        ptfsnow[3*iseas] = skj_cp_tot_seas_kw_ptf
        mapsnow[3*iseas+1] = bet_cp_tot_seas.sel(season=seasonsnow[iseas])
        pvsnow[3*iseas+1] = bet_cp_tot_seas_kw_pval
        ptfsnow[3*iseas+1] = bet_cp_tot_seas_kw_ptf
        mapsnow[3*iseas+2] = bettoskj_cp_tot_seas.sel(season=seasonsnow[iseas])
        pvsnow[3*iseas+2] = bettoskj_cp_tot_seas_kw_pval
        ptfsnow[3*iseas+2] = bettoskj_cp_tot_seas_kw_ptf

stipltdkcosnow = 0.5*np.asarray(vmaxsnow) # light/dark stip cutoff value
titlesnow = 4*['SKJ CPUE', 'BET CPUE', 'BET:SKJ CPUE']
unitsnow = 4*['[metric tons/set]', '[metric tons/set]', '']

In [ ]:
# - Set proj and define axes
fig,axes = plt.subplots(nrows=nseas, ncols=nvars, figsize=(16,9),
                        subplot_kw={'projection': ccrs.PlateCarree(central_longitude=200)})

# - Make maps pretty + plot
isp = 0;
for irow in range(nseas):
    for icol in range(nvars):
        ax = axes[irow][icol]
        exec(open('helper_scripts/create_map_bgs.py').read())
        
        mapsnow[isp].plot(
            ax=ax, transform=ccrs.PlateCarree(), cmap=cmnow,
            vmin=vminsnow[isp], vmax=vmaxsnow[isp],
            cbar_kwargs={'pad': 0.02, 'label': unitsnow[isp]})
        
        if ploteezsnow[isp]==1:
            nueezs.plot(ax=ax, transform=ccrs.PlateCarree(),
                color=eezfcnow, edgecolor=eezlcnow, linewidth=eezlwnow)
            
        if signifstipsnow[isp]==1:
            [ltcol_signiflonnow,ltcol_signiflatnow]=find_where_pval_small(
                pvsnow[isp].where(abs(mapsnow[isp])>stipltdkcosnow[isp]),
                ptfsnow[isp])
            [dkcol_signiflonnow,dkcol_signiflatnow]=find_where_pval_small(
                pvsnow[isp].where(abs(mapsnow[isp])<=stipltdkcosnow[isp]),
                ptfsnow[isp])
            ax.scatter(ltcol_signiflonnow, ltcol_signiflatnow,
                       marker=stipmarknow, linewidths=stipewnow,
                       facecolors=stipfacecolnow, edgecolors=stipedgeltcolnow,
                       s=stipsizenow, transform=ccrs.PlateCarree())
            ax.scatter(dkcol_signiflonnow, dkcol_signiflatnow,
                       marker=stipmarknow, linewidths=stipewnow,
                       facecolors=stipfacecolnow, edgecolors=stipedgedkcolnow,
                       s=stipsizenow, transform=ccrs.PlateCarree())
                    
        ax.set_xlabel(''); ax.set_ylabel('')
        ax.set_title(titlesnow[isp] + ' - ' + seasonnamesnow[irow])
        isp = isp + 1
        
fig.savefig(figpath + 'suppfig6-8_bscpue_season_' + totanomnow + '_maps.png',
            bbox_inches='tight', pad_inches = 0, dpi = 200)